# 特征

1. 5日内有且仅有一个涨停
2. 股价为60日内低位 wr < 0.6

<img src="https://images.jieyu.ai/images/202111/20211103225820.png" width="450"/>

In [60]:
from alpha.notebook import *
from alpha.core.rsi_stats import rsi30, rsiday
from alpha.utils import math_round, buy_limit_price
from numpy.lib.stride_tricks import sliding_window_view

await init_notebook()
init_jq()

In [61]:
secs = Securities().choose(['stock'])

In [62]:
def scan_buy_limit(adv_thres=0.8):
    fields = ["open", "close", "high", "low", "volume", "high_limit"]
    
    n = 20
    rec_limit_per_req = 3000
    sec_per_req = rec_limit_per_req // n
    
    batches = len(secs) // sec_per_req + 1
    
    r1 = []
    for i in range(batches):
        codes = secs[i*sec_per_req:(i+1)*sec_per_req]
        bars = jq.get_price(codes, count=n, end_date=arrow.now().date(), fields=fields, panel=False)

        for code in codes:
            bar = bars[bars.code == code]
            reach_buy_limit = (np.abs(bar['close'] - bar['high_limit']) < 1e-3).values
            if np.count_nonzero(reach_buy_limit) != 1 or np.count_nonzero(reach_buy_limit[-5:]) != 1 or reach_buy_limit[-1]:
                continue
            
            r1.append(code)
                
    # 去掉半年内大涨的个股
    n = 120
    r2 = []
    for code in r1:
        bars = jq.get_price(code, count=n, end_date=arrow.now().date(), panel=False)
        close = bars['close'].values
        
        pos_high = np.argmax(close)
        
        low = np.min(close[:pos_high+1])
        
        #print(bars.index[pos_high], close[pos_high], low, close[pos_high]/low - 1)
        
        adv = close[pos_high]/low - 1
        if adv > adv_thres:
            continue
        
        r2.append((code, Security(code).display_name, adv))
        
    return pd.DataFrame(r2, columns=['code', 'name', 'adv'])

In [63]:
df = scan_buy_limit()

In [59]:
df.sort_values('adv')

,code,name,adv
19,002627.XSHE,三峡旅游,0.000000
38,600189.XSHG,泉阳泉,0.000000
12,002289.XSHE,宇顺电子,0.025641
2,000650.XSHE,仁和药业,0.087420
46,600753.XSHG,东方银星,0.112811
...,...,...,...
32,300519.XSHE,新光药业,0.758880
29,300056.XSHE,中创环保,0.781443
7,001216.XSHE,华瓷股份,NaN
61,605507.XSHG,国邦医药,NaN
